In [1]:
import os

files = os.listdir('.')

In [2]:
import pandas as pd

full_data = pd.read_csv('data/full_dataset.csv')


In [3]:
results = {}

file_prefix = 'gpt35-reasoning'


for file in files:
    if file.startswith(file_prefix) and file.endswith('.txt') and 'multitarget' in file:
        print(file)
        with open(file, 'r') as f:
            
            data = f.readlines()
            if file_prefix!= 'gpt35-reasoning':
                temp = [x.strip() for x in data]
                
                print(len(temp))
                start = -1
                fn = file.split('.')[0]
                results[fn] = []
                for i in range(len(temp)):
                    if (temp[i].startswith('Yes') or temp[i].startswith('No') or temp[i].startswith('Unclear')):
                        if start >= 0:
                            results[fn].append('\n'.join(temp[start:i]).strip())
                        start = i
    #                 else:
    #                     print(temp[i])
                results[fn].append('\n'.join(temp[start:i]))
            else:
                results[file.split('.')[0]] = [x.strip() for x in ''.join(data).split('\n-------------------------------------\n')]
#                 results[file.split('.')[0]] = [x.strip().split('If Yes, explain why.')[-1] for x in ''.join(data).split('\n--------------\n')]

gpt35-reasoning_multitarget_nonotes.json.txt
gpt35-reasoning_multitarget.txt


In [4]:
[len(x) for x in results.values()], len(full_data)

([1, 2204], 2204)

In [5]:
del results['gpt35-reasoning_multitarget_nonotes']

results.keys()

dict_keys(['gpt35-reasoning_multitarget'])

In [6]:
from collections import Counter
import re
from nltk.tokenize import word_tokenize


hallucinations = 0
print(full_data.iloc[0]['text'])
for k, v in results.items():
    if len(full_data) < len(v):
        res = v[-len(full_data):]
    else:
        res = v
        
    print(k,res[1])
    predictions = []
    bag_of_words = []
    for r in res:
        coarse = (2 if r.strip().startswith('Yes') else (1 if r.strip().startswith('Unclear') else (0 if r.strip().startswith('No') else -1)))
        if coarse == -1:
            hallucinations += 1
            if 'Yes' in r.strip():
                coarse = 2
            elif 'Unclear' in r.strip():
                coarse = 1
            elif 'No' in r.strip():
                coarse = 0
            else:
                coarse = -1
        
        words = set(word_tokenize(r.lower()))
        
#         print(r, words)
        predictions.append(coarse)
        bag_of_words.append(words)
        
    print(Counter(predictions))
    
#     hallucinations += Counter(predictions[k])[-1]

Did he kill your sex life? Women are objects so you can just buy another one anyways.
gpt35-reasoning_multitarget Yes, this post is a violation of the policies.

1. CC_319: This post promotes hatred against the identifiable group of "Age" by expressing a strong negative sentiment towards children and young people. It implies that they are disgusting and vile, which can be seen as an expression of contempt and ill-treatment based on age.

2. HR_AB: This post is likely to expose children and young people to hatred or contempt based on their age. It devalues and dehumanizes them
Counter({0: 1548, 2: 653, -1: 3})


In [7]:
bag_of_words[1]

{"''",
 ',',
 '.',
 '1.',
 '2.',
 ':',
 '``',
 'a',
 'against',
 'age',
 'an',
 'and',
 'are',
 'as',
 'based',
 'be',
 'by',
 'can',
 'cc_319',
 'children',
 'contempt',
 'dehumanizes',
 'devalues',
 'disgusting',
 'expose',
 'expressing',
 'expression',
 'group',
 'hatred',
 'hr_ab',
 'identifiable',
 'ill-treatment',
 'implies',
 'is',
 'it',
 'likely',
 'negative',
 'of',
 'on',
 'or',
 'people',
 'policies',
 'post',
 'promotes',
 'seen',
 'sentiment',
 'strong',
 'that',
 'the',
 'their',
 'them',
 'they',
 'this',
 'to',
 'towards',
 'vile',
 'violation',
 'which',
 'yes',
 'young'}

In [8]:
list(results.values())[0][42]

'Yes, this post is a violation of the policies.\n\n1. It violates CC_319 as it communicates a statement that promotes hatred against immigrants, which is likely to lead to a breach of the peace.\n2. It violates HR_BC as it displays a statement that indicates discrimination and exposes immigrants to hatred or contempt based on their immigration status.\n3. It violates HR_NW as it expresses discrimination and incites others to discriminate against immigrants based on their nationality, ethnic origin, and immigration'

In [9]:
hallucinations/sum([len(predictions)])

0.0013611615245009074

In [10]:
full_data.iloc[5]['text']

'It’s what sore losers and cheechakos call Senator Dan Sullivan. It’s sickening that women have the right to vote. Women are sore losers.'

In [11]:

legal_defs = pd.read_csv('legal definitions.csv')
legal_labels = legal_defs['promptName'].to_list()
defs = pd.read_csv('data/definitions.csv')

In [12]:
legal_labels

['CC_318',
 'CC_319',
 'HR_AB',
 'HR_BC',
 'HR_NW',
 'HR_SA',
 'TOS_MET',
 'TOS_RED',
 'TOS_TK',
 'TOS_TWT',
 'TOS_YT']

In [13]:
import numpy as np

from sklearn.metrics import f1_score, recall_score, precision_score, hamming_loss
fine = []
pred = []

scores = [[],[],[]]
for i in range(len(legal_labels)):
    temp = []
    
#     for j in range(len(bag_of_words)):
#         r = bag_of_words[j]
#         if legal_labels[i] in r:
#             temp.append(1)
#         else:
#             temp.append(0)
    temp = [(1 if legal_labels[i].lower() in x else 0) for x in bag_of_words]
    pred.append(temp)        

    finegrained_lab = [(1 if full_data.iloc[j]['label'] == 'Meaning unclear' 
                        else (2 if (not pd.isna(full_data.iloc[j]['finegrained']) and legal_labels[i].lower() in full_data.iloc[j]['finegrained'].lower()) 
                         else 0)) 
                       for j in range(len(full_data))]



    fine.append([(1 if x > 1 else 0) for x in finegrained_lab])
    print(Counter(temp))
#         print(f1_score(finegrained_lab, predictions[r], average='micro'))
#         print(f1_score(finegrained_lab, predictions[r], average='macro'))



Counter({0: 2109, 1: 95})
Counter({0: 1659, 1: 545})
Counter({0: 1727, 1: 477})
Counter({0: 1972, 1: 232})
Counter({0: 2140, 1: 64})
Counter({0: 2169, 1: 35})
Counter({0: 2119, 1: 85})
Counter({0: 2142, 1: 62})
Counter({0: 2111, 1: 93})
Counter({0: 2107, 1: 97})
Counter({0: 2125, 1: 79})


In [14]:
results.keys()

dict_keys(['gpt35-reasoning_multitarget'])

In [15]:
coarse = [[] for _ in range(len(full_data))]

mapping = {'Violates': 2, 'Meaning unclear': 1, 'Does not violate': 0}

coarse_lab = [mapping[x] for x in full_data['label'].to_list()]

# for r in predictions.keys():
# name = '_'.join(r.split('_')[-2:]).split('.')[0]
# print(name)
# if name in legal_labels:
#     print(len(predictions))
#     for i in range(len(predictions)):
#         coarse[i].append(predictions[i])
            

            
coarse_pred = predictions

# coarse_pred = [(1 if x[1] > 4 else sorted(x.items(), key=lambda x: x[1], reverse=True)[0][0]) for x in coarse]
coarse_pred

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,


In [16]:
f1_score(coarse_lab, coarse_pred, average='macro'), precision_score(coarse_lab, coarse_pred, average='macro'), recall_score(coarse_lab, coarse_pred, average='macro'), f1_score(coarse_lab, coarse_pred, average='micro')

/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.315280105763461,
 0.3099219563058197,
 0.43723365589932306,
 0.7699637023593466)

In [25]:
rand = np.random.randint(3, size=len(coarse_pred))

f1_score(coarse_lab, rand, average='macro'), precision_score(coarse_lab, rand, average='macro'), recall_score(coarse_lab, rand, average='macro'), f1_score(coarse_lab, rand, average='micro')

(0.22255584152487354,
 0.33807799079794143,
 0.3935956264292096,
 0.338929219600726)

In [22]:
fine = np.transpose(np.array(fine))
pred = np.transpose(np.array(pred))

rand = np.random.randint(2, size=pred.shape)

In [19]:
f1_score(fine, pred, average='macro'), precision_score(fine, pred, average='macro'), recall_score(fine, pred, average='macro'), f1_score(fine, pred, average='micro')

/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.15837615752601894,
 0.13381075565978529,
 0.2693363240005093,
 0.2183730281472317)

In [23]:
f1_score(fine, rand, average='macro'), precision_score(fine, rand, average='macro'), recall_score(fine, rand, average='macro'), f1_score(fine, rand, average='micro')

/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.04284151576735985,
 0.05641996514274191,
 0.0372862832368277,
 0.10153412880752982)

In [ ]:
import json

with open('words.json', 'w') as f:
    f.write(json.dumps(list(words)))

In [ ]:
groups = {
         'Race (colour, creed)': ['actual and perceived race', 'colour', 'creed', 'race', 'race or perceived race'],
         'National or ethnic origin (nationality, ethnicity, ancestry)': ['ancestry', 'national', 'ethnic origin', 'ethnicity', 'place of origin', 'national origin', 'national or ethnic origin', 'nationality', 'indigenous identity'],
         'Political belief/association': ['political association', 'political belief'],
         'Sex/Gender identity or expression': ['gender', 'gender identity', 'gender identity and expression', 'gender identity or expression', 'sex', 'sex/gender'],
         'Religion/Religious beliefs': ['religion', 'religious affiliation','religious beliefs' 'religion/religious'],
         'Sexual orientation': ['sexual orientation'],
         'Social condition': ['social condition'],
         'Immigration status': ['immigration status', 'immigration'],
         'Source of income': ['source of income'],
         'Age': ['age'],
         'Physical or mental disability': ['physical or mental disability', 'mental or physical disability', 'physical disability', 'mental disability', 'disability', 'pregnancy or disability', 'adhd'],
         'Family affiliation': ['family affiliation', 'caste'],
         'Conviction that is subject to a pardon or record suspension': ['conviction that is subject to a pardon or record suspension'],
         'Receipt of public assistance': ['receipt of public assistance'],
         'Serious disease': ['serious disease'],
         'Family status': ['family status'],
        'Pregnancy': ['pregnancy'],
    'Victims of a major violent event and their families/kin': ['victims of a major violent event and their kin', 'victims of a major event and their families'],
    'Veteran status': ['veteran status'],
    'Marital status': ['marital status']
}


# groups = {x: k for k, v in groups.items() for x in v}

In [ ]:
import json

with open('groups_map.json', 'r') as f:
    groups_new = json.loads(f.readline())
    


In [ ]:
groups_new

In [ ]:
from ast import literal_eval

def jaccard(set1, set2):
    return len(set1.intersection(set2))/len(set1.union(set2))

scores = []

for i in range(len(bag_of_words)):
    y = set((literal_eval(full_data.iloc[i]['target_group']) if not pd.isna(full_data.iloc[i]['target_group']) else []))
   
    y_hat = set(([groups_new[x] for x in bag_of_words[i] if x in groups_new] if coarse_pred[i] == 2 else []))
    print(y, y_hat)
    if len(y) != 0 and len(y_hat) != 0:
        scores.append(jaccard(y, y_hat))
        print(scores[-1])

In [ ]:
sum(scores)/len(scores)

In [ ]:
cleaned_groups = [set(literal_eval(x)) for x in legal_defs['protected_groups_cleaned'].to_list()]

In [ ]:
set.intersection(*cleaned_groups)

In [ ]:
groups_new_new = {}

for k, v in groups_new.items():
    if v in groups_new_new:
        groups_new_new[v].append(k)
    else:
        groups_new_new[v] = [k]
groups_new_new

In [ ]:
import requests
from tqdm.notebook import tqdm


base_url = 'http://api.conceptnet.io/'

roots = set([x for y in groups.values() for x in y])
# to_search = {}

for r in tqdm(word_freq):
    if r[0] in groups_new:
        pass
    else:
        res = requests.get(base_url + '/c/en/{0}'.format(r[0].replace(' ', '_'))).json()

        new_words = set([x['start']['label'] for x in res['edges']] + [x['end']['label'] for x in res['edges']])

        if len(new_words.intersection(roots)) > 0:
            print(r)
            intersect = new_words.intersection(roots)
            print(intersect)
            for i in list(intersect):
                if i in to_search:
                    to_search[i].append(r[0])
                else:
                    to_search[i] = [r[0]]
#     print([(x['start']['label'], x['end']['label']) for x in res['edges'] if x['end']['language'] == 'en' and x['start']['language'] == 'en' ])

In [ ]:
import copy
groups = copy.deepcopy(groups_new)
groups_new = {}

for k, v in groups.items():
    print(k)
    groups_new[k] = v
    if k in to_search:
        print(to_search)
        for w in to_search[k]:
            groups_new[w] = v

In [ ]:
import nltk
from nltk.corpus import stopwords
 
words_count = Counter([x for y in bag_of_words for x in y])

stp = set(stopwords.words('english'))
word_freq = [x for x in words_count.items() if x[0] not in stp]

In [ ]:
sorted(word_freq, key=lambda x: x[0])